# Step 1: Load the Dataset


In [ ]:
import pandas as pd

#### Load the unstructured dataset (wildfire data)

In [ ]:
unstructured_file = "cleaned_unstructured_data.csv"  
try:
    unstructured_df = pd.read_csv(unstructured_file)
except FileNotFoundError:
    print(f"Error: Unstructured data file '{unstructured_file}' not found.")
    exit()


#### Load the structured dataset (weather and fire metadata)

In [ ]:
structured_file = "structured.csv" 
try:
    structured_df = pd.read_csv(structured_file)
except FileNotFoundError:
    print(f"Error: Structured data file '{structured_file}' not found.")
    exit()

# Print columns for debugging
print("Unstructured dataset columns:", unstructured_df.columns.tolist())
print("Structured dataset columns:", structured_df.columns.tolist())

# Step 2: Prepare the Datasets for Merging

#### Verify required columns in unstructured dataset

In [ ]:
unstructured_required = ["latitude", "longitude", "acq_date", "frp", "bright_ti4", "confidence", "daynight"]
missing_unstructured = [col for col in unstructured_required if col not in unstructured_df.columns]
if missing_unstructured:
    print(f"Error: Missing required columns in unstructured dataset: {missing_unstructured}")
    exit()

#### Verify required columns in structured dataset

In [ ]:
structured_required = ["latitude", "longitude", "disc_clean_date", "Temp_cont", "Temp_pre_7", "Temp_pre_15", "Temp_pre_30",
                       "Prec_cont", "Prec_pre_7", "Prec_pre_15", "Prec_pre_30",
                       "Wind_cont", "Wind_pre_7", "Wind_pre_15", "Wind_pre_30",
                       "Hum_cont", "Hum_pre_7", "Hum_pre_15", "Hum_pre_30", "state"]
missing_structured = [col for col in structured_required if col not in structured_df.columns]
if missing_structured:
    print(f"Error: Missing required columns in structured dataset: {missing_structured}")
    exit()


#### Convert date columns to datetime

In [ ]:
try:
    unstructured_df["acq_date"] = pd.to_datetime(unstructured_df["acq_date"])
except ValueError as e:
    print(f"Error converting 'acq_date' in unstructured dataset to datetime: {e}")
    print("Sample values:", unstructured_df["acq_date"].head().tolist())
    exit()

try:
    structured_df["disc_clean_date"] = pd.to_datetime(structured_df["disc_clean_date"])
except ValueError as e:
    print(f"Error converting 'disc_clean_date' in structured dataset to datetime: {e}")
    print("Sample values:", structured_df["disc_clean_date"].head().tolist())
    exit()


#### Extract year and month for matching

In [ ]:
unstructured_df["year"] = unstructured_df["acq_date"].dt.year
unstructured_df["month"] = unstructured_df["acq_date"].dt.month
structured_df["year"] = structured_df["disc_clean_date"].dt.year
structured_df["month"] = structured_df["disc_clean_date"].dt.month

#### Round latitude and longitude to 1 decimal place to increase matches

In [ ]:
unstructured_df["latitude"] = unstructured_df["latitude"].round(1)
unstructured_df["longitude"] = unstructured_df["longitude"].round(1)
structured_df["latitude"] = structured_df["latitude"].round(1)
structured_df["longitude"] = structured_df["longitude"].round(1)

# Step 3: Merge the Datasets on year, month, latitude, and longitude

In [ ]:
merged_df = pd.merge(
    unstructured_df,
    structured_df,
    on=["year", "month", "latitude", "longitude"],
    how="inner"  
)
print(f"Number of rows after merging: {len(merged_df)}")

#### Check if the merge resulted in enough rows

In [ ]:
if len(merged_df) < 10000:
    print("Error: Merged dataset has fewer than 10,000 rows. Further adjust rounding precision or check data overlap.")
    print("Sample 'year' in unstructured dataset:", unstructured_df["year"].head().tolist())
    print("Sample 'month' in unstructured dataset:", unstructured_df["month"].head().tolist())
    print("Sample 'year' in structured dataset:", structured_df["year"].head().tolist())
    print("Sample 'month' in structured dataset:", structured_df["month"].head().tolist())
    exit()

# Step 4: Rename Weather Columns

In [ ]:
column_mapping = {
    "Temp_cont": "temperature",
    "Temp_pre_7": "temperature_7d",
    "Temp_pre_15": "temperature_15d",
    "Temp_pre_30": "temperature_30d",
    "Prec_cont": "precipitation",
    "Prec_pre_7": "precipitation_7d",
    "Prec_pre_15": "precipitation_15d",
    "Prec_pre_30": "precipitation_30d",
    "Wind_cont": "wind_speed",
    "Wind_pre_7": "wind_speed_7d",
    "Wind_pre_15": "wind_speed_15d",
    "Wind_pre_30": "wind_speed_30d",
    "Hum_cont": "humidity",
    "Hum_pre_7": "humidity_7d",
    "Hum_pre_15": "humidity_15d",
    "Hum_pre_30": "humidity_30d"
}

merged_df = merged_df.rename(columns=column_mapping)

# Step 5: Select Specified Columns

In [ ]:
required_columns = [
    "acq_date", "latitude", "longitude",
    "frp", "bright_ti4", "confidence", "daynight", "state",
    "temperature", "temperature_7d", "temperature_15d", "temperature_30d",
    "precipitation", "precipitation_7d", "precipitation_15d", "precipitation_30d",
    "wind_speed", "wind_speed_7d", "wind_speed_15d", "wind_speed_30d",
    "humidity", "humidity_7d", "humidity_15d", "humidity_30d"
]


#### Since we merged on year and month, acq_date_x should exist; rename it to acq_date

In [ ]:
if "acq_date_x" in merged_df.columns:
    merged_df = merged_df.rename(columns={"acq_date_x": "acq_date"})
else:
    print("Error: 'acq_date_x' not found in merged dataset. Available columns:", merged_df.columns.tolist())
    exit()

#### Verify that all required columns are present

In [ ]:
missing_cols = [col for col in required_columns if col not in merged_df.columns]
if missing_cols:
    print(f"Error: Missing required columns in merged dataset: {missing_cols}")
    exit()

#### Create the final merged dataset with only the specified columns

In [ ]:
final_df = merged_df[required_columns]

# Step 6: Save the Merged Dataset

In [2]:
merged_file = "final_merged_wildfire_structured_data.csv"
final_df.to_csv(merged_file, index=False)
print(f"Merged dataset saved as '{merged_file}'")

C:\Users\tw9520gi\AppData\Local\Temp\ipykernel_11980\3543828047.py:8: DtypeWarning: Columns (10,16) have mixed types. Specify dtype option on import or set low_memory=False.
  unstructured_df = pd.read_csv(unstructured_file)


Unstructured dataset columns: ['latitude', 'longitude', 'brightness', 'scan', 'track', 'acq_date', 'acq_time', 'satellite', 'instrument', 'confidence', 'version', 'bright_t31', 'frp', 'daynight', 'bright_ti4', 'bright_ti5', 'type']
Structured dataset columns: ['latitude', 'longitude', 'Temp_pre_30', 'Temp_pre_15', 'Temp_pre_7', 'Temp_cont', 'Wind_pre_30', 'Wind_pre_15', 'Wind_pre_7', 'Wind_cont', 'Hum_pre_30', 'Hum_pre_15', 'Hum_pre_7', 'Hum_cont', 'Prec_pre_30', 'Prec_pre_15', 'Prec_pre_7', 'Prec_cont', 'fire_size', 'state', 'discovery_month', 'putout_time', 'disc_pre_year', 'disc_pre_month', 'stat_cause_descr', 'Unnamed: 0', 'fire_name', 'fire_size_class', 'disc_clean_date', 'cont_clean_date', 'wstation_usaf']
Number of rows after merging: 285554
Error: 'acq_date_x' not found in merged dataset. Available columns: ['latitude', 'longitude', 'brightness', 'scan', 'track', 'acq_date', 'acq_time', 'satellite', 'instrument', 'confidence', 'version', 'bright_t31', 'frp', 'daynight', 'bright